In [1]:
from collections import defaultdict
from pathlib import Path

import gurobipy as gp
import polars as pl
import pandas as pd
import numpy as np
from pandas.plotting import parallel_coordinates
from gurobipy import GRB

import os
import contextlib
import matplotlib.pyplot as plt


In [2]:
def get_data(data_dir: str, discount_factor: int = 5):
    df = pl.read_csv(
        Path(f"{data_dir}/alternatives.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000
    )
    df = df.select(
        [
            "unit",
            "schedule",
            f"npv_{discount_factor}_percent",
            "stock_0",
            "stock_5",
            "stock_25",
            "harvest_value_5",
            "harvest_value_25",
            "stock_1_0",
            "stock_1_5",
            "stock_1_25",
            "stock_2_0",
            "stock_2_5",
            "stock_2_25",
            "stock_30_0",
            "stock_30_5",
            "stock_30_25",
        ]
    )
    df_keys = pl.read_csv(
        Path(f"{data_dir}/alternatives_key.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000
    )
    df_keys = df_keys.drop("holding")
    # Split the treatments into multiple columns, to make them easier to read or whatever
    df_keys = df_keys.with_columns(
        [
            pl.when(pl.col("treatment").str.contains("_5"))
            .then(pl.col("treatment").str.extract(r"(\w+)_5", 1))
            .otherwise(pl.lit("donothing"))
            .alias("treatment_5"),
            pl.when(pl.col("treatment").str.contains("_25"))
            .then(pl.col("treatment").str.extract(r"(\w+)_25", 1))
            .otherwise(pl.lit("donothing"))
            .alias("treatment_25"),
        ]
    )

    df = df_keys.join(df, on=["unit", "schedule"], how="inner")
    return df


def get_scenario_data(data_dir: str):
    return pl.read_csv(Path(f"{data_dir}/scenario.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000)

In [ ]:
def tsrmop_asf(reference: dict, outputflag: int = 0):
    """This is the robust two stage model."""
    model = gp.Model()
    model.setParam("OutputFlag", outputflag)
    model.setParam("MIPGap", 1e-5)

    discount_factor = 5
    data_dirs = [
        "C:/MyTemp/data/two_stage_data/l1l23",
        "C:/MyTemp/data/two_stage_data/h1h23",
        "C:/MyTemp/data/two_stage_data/l1h23",
        "C:/MyTemp/data/two_stage_data/h1l23",
    ]
    n_scenarios = len(data_dirs)

    ideals = {
            "harvest5": 83939.39118553553,
            "harvest25": 425351.15530470666,
            "stock": 5985.184367164438,
            "npv": 310523.6483315748,
        }
    nadirs = {"harvest5": 0, "harvest25": 0, "stock": 260.7007001702532, "npv": 52844.33156539744}

    rho = 1e-6
    alpha = model.addVar(lb=-float("inf"))
    alphacons = [[],[],[],[]]
    normalsum = 0
    hsum5 = []
    hsum25 = []
    npvsum = []
    stocksum = []
    mvars = []
    unitsums = []
    harvest5 = []
    harvest25 = []
    stock = []
    npv = []
    x0_vars = {}
    x0_unit_sum = defaultdict(float)

    for i in range(n_scenarios):
        hsum5.append(0)
        hsum25.append(0)
        npvsum.append(0)
        stocksum.append(0)
        mvars.append({})
        unitsums.append(defaultdict(float))

        # df = get_data(data_dirs[i], discount_factor=discount_factor)
        df = get_scenario_data(data_dirs[i])

        for row in df.iter_rows(named=True):
            if (row["unit"], row["treatment_5"]) not in x0_vars:
                x0_vars[(row["unit"], row["treatment_5"])] = model.addVar(vtype=GRB.BINARY)
                x0_unit_sum[row["unit"]] += x0_vars[(row["unit"], row["treatment_5"])]
            new_var = model.addVar(vtype=GRB.BINARY)
            mvars[i][(row["unit"], row["treatment_5"], row["treatment_25"])] = new_var
            hsum5[i] += new_var * row["harvest_value_5"]
            hsum25[i] += new_var * row["harvest_value_25"]
            npvsum[i] += new_var * row[f"npv_{discount_factor}_percent"]
            stocksum[i] += new_var * row["stock_25"]
            unitsums[i][(row["unit"], row["treatment_5"])] += new_var

        for key in unitsums[i]:
            model.addConstr(unitsums[i][key] == x0_vars[key])

        harvest5.append(model.addVar())
        model.addConstr(harvest5[i] == hsum5[i])
        harvest25.append(model.addVar())
        model.addConstr(harvest25[i] == hsum25[i])
        stock.append(model.addVar())
        model.addConstr(stock[i] == stocksum[i])
        npv.append(model.addVar())
        model.addConstr(npv[i] == npvsum[i])

        objectives = {"harvest5": harvest5[i], "harvest25": harvest25[i], "stock": stock[i], "npv": npv[i]}

        normalsum = 0
        for obj in objectives:  # noqa: PLC0206
            normalsum += (objectives[obj] - reference[obj]) / (nadirs[obj] - ideals[obj])

        for obj in objectives:  # noqa: PLC0206
            alphacons[i].append(
                model.addConstr(
                    alpha >= (objectives[obj] - reference[obj]) / (nadirs[obj] - ideals[obj]) + rho * normalsum
                )
            )
            ##normalsum += objectives[obj] / (nadirs[obj] - ideals[obj])
        # model.addConstr(alpha >= -npv[i] + rho*normalsum)

    for unitsum in x0_unit_sum.values():
        model.addConstr(unitsum == 1)

    # model.setObjective(alpha + rho * normalsum, sense=GRB.MINIMIZE)
    model.setObjective(alpha, sense=GRB.MINIMIZE)
    model.optimize()
    worst_case = 0
    worst_value = -1e20
    for i in range(n_scenarios):
        for j in range(len(objectives)):
            print(alpha-alphacons[i][j].Slack)
            print(alphacons[i][j].Slack)
            if alphacons[i][j].Slack >= worst_value:
                worst_case = i+1
                worst_value = alphacons[i][j].Slack


    return (model, harvest5, harvest25, stock, npv, alpha, mvars, worst_case)


In [4]:
def rmop_asf(reference: dict, outputflag: int=0):
    """This is the robust single stage model."""
    model = gp.Model()
    model.setParam('OutputFlag', outputflag)
    model.setParam("MIPGap", 1e-5)

    discount_factor = 5
    data_dirs = [
        "C:/MyTemp/data/two_stage_data/l1l23",
        "C:/MyTemp/data/two_stage_data/h1h23",
        "C:/MyTemp/data/two_stage_data/l1h23",
        "C:/MyTemp/data/two_stage_data/h1l23",
    ]
    n_scenarios = len(data_dirs)

    ideals = {
            "harvest5": 83939.39118553553,
            "harvest25": 425351.15530470666,
            "stock": 5985.184367164438,
            "npv": 310523.6483315748,
        }
    nadirs = {"harvest5": 0, "harvest25": 0, "stock": 260.7007001702532, "npv": 52844.33156539744}

    rho = 1e-6
    alpha = model.addVar(lb=-float("inf"))
    alphacons = [[],[],[],[]]
    normalsum = 0
    hsum5 = []
    hsum25 = []
    npvsum = []
    stocksum = []
    mvars = []
    unitsums = []
    harvest5 = []
    harvest25 = []
    stock = []
    npv = []
    x0_vars = {}
    x0_unit_sum = defaultdict(float)

    for i in range(n_scenarios):
        hsum5.append(0)
        hsum25.append(0)
        npvsum.append(0)
        stocksum.append(0)
        mvars.append({})
        unitsums.append(defaultdict(float))

        # df = get_data(data_dirs[i], discount_factor=discount_factor)
        df = get_scenario_data(data_dirs[i])

        for row in df.iter_rows(named=True):
            #if (row["unit"], row["treatment_5"]) not in x0_vars:
            #    x0_vars[(row["unit"], row["treatment_5"])] = model.addVar(vtype=GRB.BINARY)
            #    x0_unit_sum[row["unit"]] += x0_vars[(row["unit"], row["treatment_5"])]
            if (row["unit"], row["treatment"]) not in x0_vars:
                x0_vars[(row["unit"], row["treatment"])] = model.addVar(vtype=GRB.BINARY)
                x0_unit_sum[row["unit"]] += x0_vars[(row["unit"], row["treatment"])]
            new_var = x0_vars[(row["unit"], row["treatment"])]
            mvars[i][(row["unit"], row["treatment_5"], row["treatment_25"])] = new_var
            hsum5[i] += new_var * row["harvest_value_5"]
            hsum25[i] += new_var * row["harvest_value_25"]
            npvsum[i] += new_var * row[f"npv_{discount_factor}_percent"]
            stocksum[i] += new_var * row["stock_25"]
            #unitsums[i][(row["unit"], row["treatment_5"])] += new_var

        #for key in unitsums[i]:
        #    model.addConstr(unitsums[i][key] == x0_vars[key])

        harvest5.append(model.addVar())
        model.addConstr(harvest5[i] == hsum5[i])
        harvest25.append(model.addVar())
        model.addConstr(harvest25[i] == hsum25[i])
        stock.append(model.addVar())
        model.addConstr(stock[i] == stocksum[i])
        npv.append(model.addVar())
        model.addConstr(npv[i] == npvsum[i])

        objectives = {"harvest5": harvest5[i], "harvest25": harvest25[i], "stock": stock[i], "npv": npv[i]}

        normalsum = 0
        for obj in objectives:  # noqa: PLC0206
            normalsum += (objectives[obj] - reference[obj]) / (nadirs[obj] - ideals[obj])

        for obj in objectives:  # noqa: PLC0206
            alphacons[i].append(
                model.addConstr(
                    alpha >= (objectives[obj] - reference[obj]) / (nadirs[obj] - ideals[obj]) + rho * normalsum
                )
            )
            ##normalsum += objectives[obj] / (nadirs[obj] - ideals[obj])
        # model.addConstr(alpha >= -npv[i] + rho*normalsum)

    for unitsum in x0_unit_sum.values():
        model.addConstr(unitsum == 1)

    # model.setObjective(alpha + rho * normalsum, sense=GRB.MINIMIZE)
    model.setObjective(alpha, sense=GRB.MINIMIZE)
    model.optimize()
    worst_case = 0
    worst_value = -1e20
    for i in range(n_scenarios):
        for j in range(len(objectives)):
            # print(alphacons[i][j].Slack)
            if alphacons[i][j].Slack >= worst_value:
                worst_case = i+1
                worst_value = alphacons[i][j].Slack
    return (model, harvest5, harvest25, stock, npv, alpha, mvars, worst_case) 

In [5]:
reference = {
            "harvest5": -1183939.39118553553,
            "harvest25": -11417274.83027785446,
            "stock": -115914.54505892277,
            "npv": 11308305.7407073733,
        }


(model, harvest5, harvest25, stock, npv, alpha, mvars, worst_case) = tsrmop_asf(reference=reference,outputflag=1)
#(model, harvest5, harvest25, stock, npv, alpha) = rmop_asf(reference=reference)


print(model.ObjVal)
print("-------------------------------------")
print(f"Worst scenario ={worst_case}")
print("-------------------------------------")
print("Harvest 5")
print(harvest5[0].x)
print(harvest5[1].x)
print(harvest5[2].x)
print(harvest5[3].x)
print("harvest 25")
print(harvest25[0].x)
print(harvest25[1].x)
print(harvest25[2].x)
print(harvest25[3].x)
print("stock")
print(stock[0].x)
print(stock[1].x)
print(stock[2].x)
print(stock[3].x)
print("NPV")
print(npv[0].x)
print(npv[1].x)
print(npv[2].x)
print(npv[3].x)


Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-18
Set parameter MIPGap to value 1e-05
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11+.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1245U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 274 rows, 992 columns and 4351 nonzeros
Model fingerprint: 0x7d87511f
Variable types: 17 continuous, 975 integer (975 binary)
Coefficient statistics:
  Matrix range     [2e-12, 1e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 101 rows and 267 columns
Presolve time: 0.01s
Presolved: 173 rows, 725 columns, 2126 nonzeros
Variable types: 4 continuous, 721 integer (721 binary)
Found heuristic solution: objective 43.4071964


In [6]:
labels = ["harvest5", "harvest25", "npv", "stock"]
namedict = {"harvest5": "Income 0", "harvest25": "Income 20", "stock": "Final timber volume", "npv": "NPV"}
ideals = {
        "harvest5": 83939.39118553553,
        "harvest25": 425351.15530470666,
        "stock": 5985.184367164438,
        "npv": 310523.6483315748,
    }
nadirs = {"harvest5": 0, "harvest25": 0, "stock": 260.7007001702532, "npv": 52844.33156539744}

for xi in range(3):
    for yi in range(xi + 1, 4):
        xlabel = labels[xi]
        ylabel = labels[yi]

        objective_values = {label: [] for label in labels}

        for i in range(21):
            weight = i / 20
            """
            reference = {
                    "harvest5": weight*ideals["harvest5"]+(1-weight)*nadirs["harvest5"],
                    "harvest25": (1-weight)*ideals["harvest25"]+weight*nadirs["harvest25"],
                    "stock": nadirs["stock"],
                    "npv": nadirs["npv"],
                }
                """
            reference = {}
            for label in labels:
                if label == xlabel:
                    reference[label] = weight * ideals[label] + (1 - weight) * nadirs[label]
                elif label == ylabel:
                    reference[label] = (1 - weight) * ideals[label] + weight * nadirs[label]
                else:
                    reference[label] = -ideals[label]

            (model, harvest5, harvest25, stock, npv, alpha, mvars, worst_case) = tsrmop_asf(reference=reference)
            print(f"Worst scenario ={worst_case}")
            # (model, harvest5, harvest25, stock, npv, alpha) = rmop_asf(reference=reference)
            results = {"harvest5": harvest5, "harvest25": harvest25, "stock": stock, "npv": npv}

            for label in labels:
                objective_values[label].append(min(var.x for var in results[label]))

            if np.isclose(min(var.x for var in results[xlabel]), ideals[xlabel]):
                break

        """
            print(model.ObjVal)
            print("Harvest 5")
            print(harvest5[0].x)
            print(harvest5[1].x)
            print(harvest5[2].x)
            print(harvest5[3].x)
            print("harvest 25")
            print(harvest25[0].x)
            print(harvest25[1].x)
            print(harvest25[2].x)
            print(harvest25[3].x)
            print("stock")
            print(stock[0].x)
            print(stock[1].x)
            print(stock[2].x)
            print(stock[3].x)
            print("NPV")
            print(npv[0].x)
            print(npv[1].x)
            print(npv[2].x)
            print(npv[3].x)
            print()
        """

        data = np.array([objective_values[label] for label in labels]).transpose()

        ideal_vals = np.array([ideals[label] for label in labels])
        nadir_vals = np.array([nadirs[label] for label in labels])
        normed = (data - nadir_vals) / (ideal_vals - nadir_vals)

        # Plot each row as a line
        for row in normed:
            plt.plot(range(data.shape[1]), row)

        plt.xticks(range(normed.shape[1]), [namedict[label] for label in labels])

        # plt.plot(x, y, marker='.', linestyle='None')
        # plt.title("Pareto frontier")
        plt.savefig(f"figures/pcp_{xlabel}_{ylabel}.png", dpi=300, bbox_inches="tight")
        # plt.show()
        plt.close()

        plt.plot(objective_values[xlabel], objective_values[ylabel], marker=".", linestyle="None")
        plt.xlabel(namedict[xlabel])
        plt.ylabel(namedict[ylabel])

        # plt.title("Pareto frontier")
        plt.savefig(f"figures/scatter_{xlabel}_{ylabel}.png", dpi=300, bbox_inches="tight")
        # plt.show()
        plt.close()

Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =4
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =2
Worst scenario =2
Worst scenario =2
Worst scenario =4
Worst scenario =4
Worst scenario =2
Worst scenario =2
Worst scenario =4
Worst scenario =4
Worst scenario =3
Worst scenario =4
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scenario =3
Worst scen

In [7]:

rand_rmop = []
rand_tsrmop = []
rand_alphadiff = []

for i in range(101):
    arr = np.random.rand(4)
    reference = {
        "harvest5": arr[0] * ideals["harvest5"] + (1 - arr[0]) * nadirs["harvest5"],
        "harvest25": arr[1] * ideals["harvest25"] + (1 - arr[1]) * nadirs["harvest25"],
        "stock": arr[2] * ideals["stock"] + (1 - arr[2]) * nadirs["stock"],
        "npv": arr[3] * ideals["npv"] + (1 - arr[3]) * nadirs["npv"],
    }
    (model, harvest5, harvest25, stock, npv, alpha, mvars, worst_case)  = tsrmop_asf(reference)
    rand_tsrmop.append(
        [
            min(var.x for var in harvest5),
            min(var.x for var in harvest25),
            min(var.x for var in npv),
            min(var.x for var in stock),
        ]
    )
    rand_alphadiff.append(-alpha.x)
    (model, harvest5, harvest25, stock, npv, alpha, mvars, worst_case)  = rmop_asf(reference)
    rand_rmop.append(
        [
            min(var.x for var in harvest5),
            min(var.x for var in harvest25),
            min(var.x for var in npv),
            min(var.x for var in stock),
        ]
    )
    rand_alphadiff[-1] += alpha.x
    print(i)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [8]:
import statistics
print(statistics.mean(rand_alphadiff))
print(max(rand_alphadiff))

0.005156049243875246
0.017443176766215818


In [9]:
reference = ideals


(model, harvest5, harvest25, stock, npv, alpha, mvars, worst_case)  = tsrmop_asf(reference=reference,outputflag=0)
#(model, harvest5, harvest25, stock, npv, alpha) = rmop_asf(reference=reference)


# print(model.ObjVal)
print("Harvest 5")
print(harvest5[worst_case-1].x)
print("harvest 25")
print(harvest25[worst_case-1].x)
print("stock")
print(stock[worst_case-1].x)
print("NPV")
print(npv[worst_case-1].x)

print("-------------------------------------")
print(f"Worst scenario ={worst_case}")
print("-------------------------------------")
print("Harvest 5")
print(harvest5[0].x)
print(harvest5[1].x)
print(harvest5[2].x)
print(harvest5[3].x)
print("harvest 25")
print(harvest25[0].x)
print(harvest25[1].x)
print(harvest25[2].x)
print(harvest25[3].x)
print("stock")
print(stock[0].x)
print(stock[1].x)
print(stock[2].x)
print(stock[3].x)
print("NPV")
print(npv[0].x)
print(npv[1].x)
print(npv[2].x)
print(npv[3].x)

Harvest 5
39531.31372379564
harvest 25
197137.7084990426
stock
2915.9639302305727
NPV
181569.0951736005
-------------------------------------
Worst scenario =3
-------------------------------------
Harvest 5
39531.31372379564
39531.31372379564
39531.31372379564
39531.31372379564
harvest 25
205689.49616163038
229554.2044999958
197137.7084990426
217696.79524565753
stock
2929.052359934927
2926.2568826136194
2915.9639302305727
2921.0048984899536
NPV
183099.16157583057
172637.15103930482
181569.0951736005
177995.98824049253


In [20]:
reference = {
    "harvest5": 0,
    "harvest25": 0,
    "stock": 0,
    "npv": 310524,
}

(model, harvest5, harvest25, stock, npv, alpha, mvars, worst_case)  = tsrmop_asf(reference=reference,outputflag=0)
#(model, harvest5, harvest25, stock, npv, alpha) = rmop_asf(reference=reference)


# print(model.ObjVal)
print("Harvest 5")
print(harvest5[worst_case-1].x)
print("harvest 25")
print(harvest25[worst_case-1].x)
print("stock")
print(stock[worst_case-1].x)
print("NPV")
print(npv[worst_case-1].x)

print("-------------------------------------")
print(f"Worst scenario ={worst_case}")
print("-------------------------------------")
print("Harvest 5")
print(harvest5[0].x)
print(harvest5[1].x)
print(harvest5[2].x)
print(harvest5[3].x)
print("harvest 25")
print(harvest25[0].x)
print(harvest25[1].x)
print(harvest25[2].x)
print(harvest25[3].x)
print("stock")
print(stock[0].x)
print(stock[1].x)
print(stock[2].x)
print(stock[3].x)
print("NPV")
print(npv[0].x)
print(npv[1].x)
print(npv[2].x)
print(npv[3].x)

Harvest 5
3243.622737551953
harvest 25
424312.7274145438
stock
264.5185704334657
NPV
310515.35252537666
-------------------------------------
Worst scenario =3
-------------------------------------
Harvest 5
3243.622737551953
3243.622737551953
3243.622737551953
3243.622737551953
harvest 25
433832.8630816395
472801.26748899464
424312.7274145438
461148.46376199904
stock
365.6316574362934
683.0951812079145
264.5185704334657
906.9404687207646
NPV
318000.7086157016
322574.2688592497
310515.35252537666
312516.5216721112


In [11]:
reference = {
    "harvest5": 20000,
    "harvest25": 0,
    "stock": 1000,
    "npv": 250000,
}

(model, harvest5, harvest25, stock, npv, alpha, mvars, worst_case)  = tsrmop_asf(reference=reference,outputflag=0)
#(model, harvest5, harvest25, stock, npv, alpha) = rmop_asf(reference=reference)


# print(model.ObjVal)
print("Harvest 5")
print(harvest5[worst_case-1].x)
print("harvest 25")
print(harvest25[worst_case-1].x)
print("stock")
print(stock[worst_case-1].x)
print("NPV")
print(npv[worst_case-1].x)

print("-------------------------------------")
print(f"Worst scenario ={worst_case}")
print("-------------------------------------")
print("Harvest 5")
print(harvest5[0].x)
print(harvest5[1].x)
print(harvest5[2].x)
print(harvest5[3].x)
print("harvest 25")
print(harvest25[0].x)
print(harvest25[1].x)
print(harvest25[2].x)
print(harvest25[3].x)
print("stock")
print(stock[0].x)
print(stock[1].x)
print(stock[2].x)
print(stock[3].x)
print("NPV")
print(npv[0].x)
print(npv[1].x)
print(npv[2].x)
print(npv[3].x)

Harvest 5
24822.6
harvest 25
324964.81039709284
stock
1326.6310911256508
NPV
265493.63293289876
-------------------------------------
Worst scenario =3
-------------------------------------
Harvest 5
24822.6
24822.6
24822.6
24822.6
harvest 25
326950.25192241877
369849.27813453553
324964.81039709284
369439.78019142954
stock
1499.9608167668114
1496.5213536708375
1326.6310911256508
1592.02310428232
NPV
279133.40061280085
274933.40514962305
265493.63293289876
274799.3644387467


In [12]:
reference = {
    "harvest5": 30000,
    "harvest25": 0,
    "stock": 1000,
    "npv": 280000,
}

(model, harvest5, harvest25, stock, npv, alpha, mvars, worst_case)  = tsrmop_asf(reference=reference,outputflag=0)
#(model, harvest5, harvest25, stock, npv, alpha) = rmop_asf(reference=reference)


# print(model.ObjVal)
print("Harvest 5")
print(harvest5[worst_case-1].x)
print("harvest 25")
print(harvest25[worst_case-1].x)
print("stock")
print(stock[worst_case-1].x)
print("NPV")
print(npv[worst_case-1].x)

print("-------------------------------------")
print(f"Worst scenario ={worst_case}")
print("-------------------------------------")
print("Harvest 5")
print(harvest5[0].x)
print(harvest5[1].x)
print(harvest5[2].x)
print(harvest5[3].x)
print("harvest 25")
print(harvest25[0].x)
print(harvest25[1].x)
print(harvest25[2].x)
print(harvest25[3].x)
print("stock")
print(stock[0].x)
print(stock[1].x)
print(stock[2].x)
print(stock[3].x)
print("NPV")
print(npv[0].x)
print(npv[1].x)
print(npv[2].x)
print(npv[3].x)

Harvest 5
27800.3233121971
harvest 25
356302.23765202006
stock
864.2750958793922
NPV
273288.9837401398
-------------------------------------
Worst scenario =3
-------------------------------------
Harvest 5
27800.3233121971
27800.3233121971
27800.3233121971
27800.323312197102
harvest 25
337469.7832285571
400677.8474923964
356302.23765202006
395927.04196364124
stock
1272.1156978663214
1158.5955929061108
864.2750958793922
1208.4429192510825
NPV
282124.7052678393
275978.8100724469
273288.9837401398
275257.4918004624


In [13]:
for scenario in range(4):
    print(f"Scenario {scenario+1}")
    for key, value in mvars[scenario].items():
        if value.x != 0:
            print(f"Stand: {key[0]-43124559} First action: {key[1]} Second action: {key[2]}")

Scenario 1
Stand: 0 First action: donothing Second action: donothing
Stand: 1 First action: donothing Second action: above
Stand: 2 First action: donothing Second action: above
Stand: 3 First action: donothing Second action: even
Stand: 4 First action: donothing Second action: clearcut
Stand: 5 First action: donothing Second action: donothing
Stand: 6 First action: even Second action: above
Stand: 7 First action: donothing Second action: clearcut
Stand: 8 First action: above Second action: clearcut
Stand: 9 First action: donothing Second action: donothing
Stand: 10 First action: clearcut Second action: first
Stand: 11 First action: donothing Second action: below
Stand: 12 First action: even Second action: clearcut
Stand: 13 First action: above Second action: clearcut
Stand: 14 First action: donothing Second action: donothing
Stand: 15 First action: above Second action: below
Stand: 16 First action: donothing Second action: clearcut
Stand: 17 First action: donothing Second action: clear

In [14]:
'''
tsr_f1_arvot = [1e6]*101
tsr_f2_arvot = [1e6]*101
r_f1_arvot = [1e6]*101
r_f2_arvot = [1e6]*101
diff_f1 = [0]*101
diff_f2 = [0]*101
alphas = [0]*101
# heitetään tulosteet roskiin
with open(os.devnull, 'w') as f, contextlib.redirect_stdout(f), contextlib.redirect_stderr(f):
    for i in range(0,101):
        reference = {
            "harvest5": -1e5,
            "harvest25": -1e5,
            "stock": 5914.54505892277*i/100+(100-i)/100*277.3128133588118,
            "npv": 308305.7407073733*(100-i)/100+i/100*137635.7828109703,
        }

        (model, harvest5, harvest25, stock, npv, alpha) = tsrmop_asf(reference=reference)
        for s in stock:
            tsr_f1_arvot[i] = min(s.x,tsr_f1_arvot[i])
        for n in npv:
            tsr_f2_arvot[i] = min(n.x,tsr_f2_arvot[i])

        alphas[i]=alpha

        (model, harvest5, harvest25, stock, npv, alpha) = rmop_asf(reference=reference)
        for s in stock:
            r_f1_arvot[i] = min(s.x,r_f1_arvot[i])
        for n in npv:
            r_f2_arvot[i] = min(n.x,r_f2_arvot[i])
        diff_f1[i] = (tsr_f1_arvot[i] - r_f1_arvot[i])/(5914.54505892277-277.3128133588118)
        diff_f2[i] = (tsr_f2_arvot[i] - r_f2_arvot[i])/(308305.7407073733-137635.7828109703)
        alphas[i] = alphas[i]-alpha
        
'''



'\ntsr_f1_arvot = [1e6]*101\ntsr_f2_arvot = [1e6]*101\nr_f1_arvot = [1e6]*101\nr_f2_arvot = [1e6]*101\ndiff_f1 = [0]*101\ndiff_f2 = [0]*101\nalphas = [0]*101\n# heitetään tulosteet roskiin\nwith open(os.devnull, \'w\') as f, contextlib.redirect_stdout(f), contextlib.redirect_stderr(f):\n    for i in range(0,101):\n        reference = {\n            "harvest5": -1e5,\n            "harvest25": -1e5,\n            "stock": 5914.54505892277*i/100+(100-i)/100*277.3128133588118,\n            "npv": 308305.7407073733*(100-i)/100+i/100*137635.7828109703,\n        }\n\n        (model, harvest5, harvest25, stock, npv, alpha) = tsrmop_asf(reference=reference)\n        for s in stock:\n            tsr_f1_arvot[i] = min(s.x,tsr_f1_arvot[i])\n        for n in npv:\n            tsr_f2_arvot[i] = min(n.x,tsr_f2_arvot[i])\n\n        alphas[i]=alpha\n\n        (model, harvest5, harvest25, stock, npv, alpha) = rmop_asf(reference=reference)\n        for s in stock:\n            r_f1_arvot[i] = min(s.x,r_f1

In [15]:
#plt.plot(tsr_f1_arvot[40:61], tsr_f2_arvot[40:61], marker='.', linestyle='-',color='red')
#plt.plot(r_f1_arvot[40:61], r_f2_arvot[40:61], marker='.', linestyle='-',color='blue')
#plt.plot(diff_f1, diff_f2, marker='.', linestyle='None')
#plt.title("Pareto frontier")
#plt.show()
#plt.savefig('image.png')

In [19]:
reference = {
    "harvest5": 0,
    "harvest25": 0,
    "stock": 0,
    "npv": 1000000,
}

(model, harvest5, harvest25, stock, npv, alpha, mvars, worst_case)  = tsrmop_asf(reference=reference,outputflag=0)
#(model, harvest5, harvest25, stock, npv, alpha) = rmop_asf(reference=reference)


# print(model.ObjVal)
print("Harvest 5")
print(harvest5[worst_case-1].x)
print("harvest 25")
print(harvest25[worst_case-1].x)
print("stock")
print(stock[worst_case-1].x)
print("NPV")
print(npv[worst_case-1].x)
print("-------------------------------------")
print(f"Worst scenario ={worst_case}")
print("-------------------------------------")
print("Harvest 5")
print(harvest5[0].x)
print(harvest5[1].x)
print(harvest5[2].x)
print(harvest5[3].x)
print("harvest 25")
print(harvest25[0].x)
print(harvest25[1].x)
print(harvest25[2].x)
print(harvest25[3].x)
print("stock")
print(stock[0].x)
print(stock[1].x)
print(stock[2].x)
print(stock[3].x)
print("NPV")
print(npv[0].x)
print(npv[1].x)
print(npv[2].x)
print(npv[3].x)

Harvest 5
3243.622737551953
harvest 25
424490.9513276124
stock
260.7007001702532
NPV
310523.6483315748
-------------------------------------
Worst scenario =3
-------------------------------------
Harvest 5
3243.622737551953
3243.622737551953
3243.622737551953
3243.622737551953
harvest 25
415204.7476931632
472981.281089882
424490.9513276124
484964.057924659
stock
671.7671469640364
686.7775092328045
260.7007001702532
606.9994584971819
NPV
313011.3498190064
317953.3857302591
310523.6483315748
322615.0479927271
